In [ ]:
import numpy as np
from numba import jit, njit
import matplotlib.pyplot as plt
from random import randint
from sklearn.datasets import make_blobs
from utility import *
from tqdm import tqdm
from sklearn.cluster import k_means

Implementazione del local search. La funzione neighbors clustering restituisce un sottoinsieme dell'intorno della soluzione sol tramite le mosse di scambio tra due elementi e di spostamento di un elemento.

In [ ]:
@njit
def neighbors_clustering(sol, points, K, q):
    neighbors = []

    choices = np.arange(len(sol))
    orig_val = squared_inner_distance(sol, points, K)
    centroids = calc_centroids(sol, points, K)

    for i in range(q):#mossa di spostamento di un nodo
        choice = np.random.randint(len(choices))
        choices = np.delete(choices, choice)
        for k in range(K):
            new_sol = sol.copy()
            if(k != new_sol[choice]):
                new_sol[choice] = k
                new_val = orig_val - np.linalg.norm(points[choice]-centroids[sol[choice]])**2 + np.linalg.norm(points[choice]-centroids[k])**2
                neighbors.append((new_sol,new_val))


    choices1 = np.arange(len(sol))
    choices2 = np.arange(len(sol))
    for i in range(q):
        choice1 = np.random.randint(len(choices1))
        choices1 = np.delete(choices1, choice1)
        choice2 = np.random.randint(len(choices2))
        choices2 = np.delete(choices2, choice2)

        new_sol = sol.copy()
        new_sol[choice1], new_sol[choice2] = new_sol[choice2], new_sol[choice1]

        new_val = orig_val - np.linalg.norm(points[choice1]-centroids[sol[choice1]])**2 - np.linalg.norm(points[choice2]-centroids[sol[choice2]])**2 + np.linalg.norm(points[choice1]-centroids[sol[choice2]])**2 + np.linalg.norm(points[choice2]-centroids[sol[choice1]])**2
        neighbors.append((new_sol,new_val))
        
    return neighbors


@njit
def local_search(base_sol, points, K, q = 100, verbose = True):
    old_sol = base_sol
    base_val = squared_inner_distance(old_sol, points, K)
    iter = 1
    same_sol = 0

    while True:
        neighbourhood = neighbors_clustering(old_sol, points, K, q)
        best_val = squared_inner_distance(old_sol, points , K)
        best_sol = old_sol

        if verbose:
            print("Iteration number:", iter, "Valore percentuale:", best_val/base_val*100, "%")

        for sol,val in neighbourhood:
            if(val < best_val):
                best_val = val
                best_sol = sol
        
        if((best_sol == old_sol).all()):
            same_sol = same_sol + 1
            if(same_sol == 500):
                break
        else:
            same_sol = 0
            curr_sol = best_sol

        iter = iter+1
    return curr_sol

Testing dell'euristica con un istanza di 1000 punti in R^2 e un numero di cluster pari a 10

Istanza di prova della local search

In [ ]:
K = 10
N = 1000

points, centroids = make_blobs(n_samples=N, centers=K, n_features=2, random_state=np.random.randint(10))

sol = np.random.randint(K, size = N)
sol = local_search(sol, points, K, q = 500)

print("Il valore di f.obj ottenuta è: {:.5E}".format(squared_inner_distance(sol, points, K)))

printR2sol(points, sol, K)